# Restriction de (LP) - Modèle $(RLP)$

Dans ce notebook, nous définissons le modèle $(RLP)$, accompagné de sa fonction de résolution.

In [ ]:
function model_RLP(n,C_hat,W_hat,C_hat_v)
    """
    Crée la restriction du modèle (LP) aux cycles de C_hat.

    Entrées : 
        - n       : Integer - Nombre de sommets dans le graphe
        - C_hat   : Array   - Sous-espace de cycles de taille <= K
        - W_hat   : Array   - Matrice des poids de chaque cycle
        - C_hat_v : Array   - Matrice répertoriant les cycles passant par chaque sommet 

    Sortie :
        - model : JuMP model - Modèle du problème relaxé
    """
    # Vecteur des sommets
    V=Array(1:n)
    
    # Création du modèle
    model = Model(HiGHS.Optimizer); 
    set_optimizer_attribute(model,"output_flag",false)
    
    # Variables
    ## Pour tout cycle c de C_K, x[c] = 1 si le cycle c est utilisé dans la solution, 0 sinon
    @variable(model, x[1:length(C_hat)]>=0);
    
    # Fonction objectif: On souhaite maximiser la somme des coûts des transplantations
    @objective(model, Max, sum(x[c]*W_hat[c] for c in range(1,length(C_hat)) ));
    
    # Contraintes
    ## Pour chaque sommet v, on a, au plus, un cycle choisi passant par v
    @constraint(model,ct_unique[v in V],sum(x[c] for c in C_hat_v[v])<=1)

    return model;
end

In [2]:
function solve_P(model,relax,logs)
    """
    Résout le problème global (relax=0) ou sa relaxation linéaire (relax=1)

    Entrées : 
        - model : JuMP Model - Modèle à résoudre
        - relax : Integer    - Indique quel problème résoudre    
        - logs  : String     - Logs auxquels ajouter la résolution du problème
    """
    # Nous pouvons résoudre la relaxation ou le modèle avec contrainte d'intégrité.
    if relax == 0
        # Si l'on souhaite résoudre le problème avec contrainte d'intégrité
        set_integer.(model[:x])
    end 
    
    # Ajout de la résolution aux logs 
    logs *= "* Résolution de (RLP) *\n"
    timer = @timed optimize!(model);

    # Ajout de la résolution dans les logs
    logs *= "| Statut de la solution = "*string(termination_status(model))*"\n"
    logs *= "| Valeur optimale = "*string(objective_value(model))*"\n"
    
    return logs
end

solve_P (generic function with 1 method)

In [ ]:
println("Modèle (RLP) importé avec succès.")